#BUILD

###Import library

In [ ]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
import torch
from transformers import AutoTokenizer
from tensorflow.keras.layers import Concatenate
import json

###Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/model_build/train.json", "r", encoding = "utf-8") as f:
  dataset = json.load(f)

###Tokenize

In [ ]:
# Load tokenizer PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Hàm token hóa
def tokenize_data(data, tokenizer, max_length=50):
    input_ids = []

    for sample in data:
        encoded = tokenizer(
            sample, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf"
        )
        input_ids.append(encoded["input_ids"])

    return tf.concat(input_ids, axis=0)  # Chuyển thành TensorFlow Tensor

# Token hóa dữ liệu
input_ids = tokenize_data([x["prompt"] for x in dataset], tokenizer)
target_ids = tokenize_data([x["response"] for x in dataset], tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [ ]:
import numpy as np
input_ids_np = input_ids.numpy() if isinstance(input_ids, tf.Tensor) else input_ids
target_ids_np = target_ids.numpy() if isinstance(target_ids, tf.Tensor) else target_ids

### Lớp Seq2Seq với Attention

In [ ]:

class Seq2SeqAttention(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, hidden_size, **kwargs):
        super(Seq2SeqAttention, self).__init__(**kwargs)

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.encoder_lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)
        self.decoder_lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=True, return_state=True)
        self.attention = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(vocab_size, activation="softmax")

        # Lưu tham số
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size

    def call(self, inputs):
        encoder_inputs, decoder_inputs = inputs

        encoder_embedded = self.embedding(encoder_inputs)
        encoder_outputs, state_h, state_c = self.encoder_lstm(encoder_embedded)

        decoder_embedded = self.embedding(decoder_inputs)
        decoder_outputs, _, _ = self.decoder_lstm(decoder_embedded, initial_state=[state_h, state_c])

        # Áp dụng Attention
        context_vector = self.attention([decoder_outputs, encoder_outputs, encoder_outputs])
        concat_outputs = tf.concat([decoder_outputs, context_vector], axis=-1)

        outputs = self.dense(concat_outputs)
        return outputs

    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "embedding_dim": self.embedding_dim,
            "hidden_size": self.hidden_size
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)



###Training

In [ ]:
# Thông số mô hình
VOCAB_SIZE = tokenizer.vocab_size
EMBEDDING_DIM = 256
HIDDEN_SIZE = 512

# Tạo mô hình
model = Seq2SeqAttention(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_SIZE)
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [ ]:
# Xác nhận shape dữ liệu trước khi huấn luyện
print("input_ids_np shape:", input_ids_np.shape)
print("target_ids_np shape:", target_ids_np.shape)

input_ids_np shape: (5006, 50)
target_ids_np shape: (5006, 50)


In [ ]:
model.fit(
    [input_ids_np, target_ids_np[:, :-1]],  # Bỏ token cuối
    target_ids_np[:, 1:],  # Dịch phải 1 bước
    batch_size=16,
    epochs=20
)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


313/313 ━━━━━━━━━━━━━━━━━━━━ 89s 248ms/step - accuracy: 0.4816 - loss: 3.9081
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 256ms/step - accuracy: 0.6421 - loss: 1.9695
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 83s 260ms/step - accuracy: 0.7606 - loss: 1.2463
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 260ms/step - accuracy: 0.8039 - loss: 0.9342
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 260ms/step - accuracy: 0.8242 - loss: 0.7889
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 260ms/step - accuracy: 0.8425 - loss: 0.6713
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 81s 260ms/step - accuracy: 0.8526 - loss: 0.6111
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 260ms/step - accuracy: 0.8641 - loss: 0.5348
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 260ms/step - accuracy: 0.8721 - loss: 0.4943
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 260ms/step - accuracy: 0.8788 - loss: 0.4563
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 82s 261ms/step - accuracy: 0.8878 - loss: 0.4171
Epoch 12/20
313/313 ━━━━━━━━━━

In [ ]:
model.summary()

Model: "seq2_seq_attention"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 50, 256)             │      16,384,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ((None, 50, 512), (None,    │       1,574,912 │
│                                      │ 512), (None, 512))          │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ((None, 49, 512), (None,    │       1,574,912 │
│                                      │ 512), (None, 512))          │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention (Attention)                │ (None, 49, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 49, 64000)           │      65,600,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 255,401,474 (974.28 MB)

 Trainable params: 85,133,824 (324.76 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 170,267,650 (649.52 MB)

In [ ]:
# Hàm sinh phản hồi
def generate_response(prompt, tokenizer, model, max_length=150):
    encoded_prompt = tokenizer(prompt, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf")
    input_ids = encoded_prompt["input_ids"]
    attention_mask = encoded_prompt["attention_mask"]

    decoder_input = tf.convert_to_tensor([[tokenizer.bos_token_id]])
    response = []

    for _ in range(max_length):
        predictions = model([input_ids, decoder_input])
        predicted_id = tf.argmax(predictions[:, -1, :], axis=-1).numpy()[0]

        if predicted_id == tokenizer.eos_token_id:
            break

        response.append(predicted_id)
        decoder_input = tf.concat([decoder_input, tf.convert_to_tensor([[predicted_id]])], axis=1)

    return tokenizer.decode(response)




#TEST

In [ ]:
# Ví dụ sử dụng
prompt = "Đất công nghiệp là gì?"
response = generate_response(prompt, tokenizer, model)
print("Chatbot:", response)

Chatbot: Đất công nghiệp là loại đất được sử dụng cho mục đích xây dựng các công trình phục vụ hoạt động thu nhập để nhà ở xã hội như nhà hát, sử dụng vào mục đích công cộng như nhà hát, công viên nhà hát, công viên và các công trình phục vụ sản xuất công nghiệp.


#SAVE

In [ ]:
import os
import joblib  # Thư viện giúp lưu object dễ dàng

# Tạo thư mục nếu chưa có
save_dir = "/content/drive/MyDrive/model_build/saved_model"
os.makedirs(save_dir, exist_ok=True)

# Lưu mô hình
model.save(os.path.join(save_dir, "seq2seq_model.keras"))

# Lưu tokenizer
tokenizer_path = os.path.join(save_dir, "tokenizer.pkl")
joblib.dump(tokenizer, tokenizer_path)

print(" Mô hình & tokenizer đã được lưu!")
